In [1]:
packageVersion('IRdisplay')
packageVersion('IRkernel')
packageVersion('repr')

[1] ‘0.4.9000’

[1] ‘0.6’

[1] ‘0.9’

In [2]:
clean_column <- function(col) {
    col <- sapply(col, function(x) toString(x))
    col <- sapply(col, function(x) tolower(x))
    col <- sapply(col, function(x) gsub("-", "", x))
    return(col)
}

In [3]:
df = rbind(
    read.csv("data/kijiji_cars.csv"), 
    read.csv("data/ottawahonda_cars.csv"),
    read.csv("data/tonygrahamtoyota_cars.csv"),
    read.csv("data/jimtubman_cars.csv")
)

df$price <- lapply(df$price, function(x) gsub("\\$", "", x))
df$price <- lapply(df$price, function(x) gsub("\\,", "", x))
df$price <- as.integer(df$price)
    
df$age <- 2016 - df$year
df$make <- clean_column(df$make)
df$model <- clean_column(df$model)

Warning message:
In `[<-.factor`(`*tmp*`, ri, value = c(173364, 86548, 41900, 23006, : invalid factor level, NA generatedWarning message:
In `[<-.factor`(`*tmp*`, ri, value = c(5, 5, 5, 5, 6, 5, 5, 10, : invalid factor level, NA generatedWarning message:
In `[<-.factor`(`*tmp*`, ri, value = c(30279L, 53883L, 34243L, 27971L, : invalid factor level, NA generatedWarning message:
In eval(expr, envir, enclos): NAs introduced by coercionWarning message:
In eval(expr, envir, enclos): NAs introduced by coercionWarning message:
In eval(expr, envir, enclos): NAs introduced by coercionWarning message:
In eval(expr, envir, enclos): NAs introduced by coercionWarning message:
In eval(expr, envir, enclos): NAs introduced by coercionWarning message:
In eval(expr, envir, enclos): NAs introduced by coercionWarning message:
In eval(expr, envir, enclos): NAs introduced by coercionWarning message:
In eval(expr, envir, enclos): NAs introduced by coercionWarning message:
In eval(expr, envir, enclos): NAs int

In [4]:
nrow(df)

[1] 1099

In [5]:
colnames(df)

[1] "domain"      "description" "date_listed" "url"         "make"       
 [6] "title"       "year"        "kilometers"  "model"       "price"      
[11] "age"

In [9]:
eff <- read.csv("data/nrcan_fuel_ratings.csv")
eff <- eff[!duplicated(eff$title), ]
nrow(eff)

[1] 734

In [10]:
eff$make <- clean_column(eff$make)
eff$model <- clean_column(eff$model)
nrow(eff)
colnames(eff)

[1] 734

[1] "engine_size_litres"       "domain"                  
 [3] "cylinders"                "title"                   
 [5] "transmission"             "year"                    
 [7] "make"                     "ranking_overall"         
 [9] "lphkm_city"               "ranking_class"           
[11] "lphkm_hwy"                "url"                     
[13] "model_class"              "fuel"                    
[15] "estimated_fuel_cost_year" "model"

In [11]:
summary(eff)

 engine_size_litres             domain      cylinders     
 Min.   :0.900      oee.nrcan.gc.ca:734   Min.   : 3.000  
 1st Qu.:2.000                            1st Qu.: 4.000  
 Median :3.000                            Median : 6.000  
 Mean   :3.198                            Mean   : 5.695  
 3rd Qu.:3.775                            3rd Qu.: 6.000  
 Max.   :8.400                            Max.   :12.000  
                                                          
                    title      transmission      year          make          
 ACURA ILX             :  1   AS6    :167   Min.   :2016   Length:734        
 ACURA MDX SH-AWD      :  1   AS8    :147   1st Qu.:2016   Class :character  
 ACURA MDX SH-AWD ELITE:  1   AS7    : 66   Median :2016   Mode  :character  
 ACURA RDX AWD         :  1   A6     : 63   Mean   :2016                     
 ACURA RLX             :  1   M6     : 57   3rd Qu.:2016                     
 ACURA RLX HYBRID      :  1   AM7    : 49   Max.   :2016    

In [12]:
library(dplyr)
df <- dplyr::left_join(
    df, 
    eff, 
    by=c("make" = "make", "model" = "model", "year" = "year")
)  
nrow(df)
colnames(df)

Warning message:
: package ‘dplyr’ was built under R version 3.2.5
Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



[1] 1099

[1] "domain.x"                 "description"             
 [3] "date_listed"              "url.x"                   
 [5] "make"                     "title.x"                 
 [7] "year"                     "kilometers"              
 [9] "model"                    "price"                   
[11] "age"                      "engine_size_litres"      
[13] "domain.y"                 "cylinders"               
[15] "title.y"                  "transmission"            
[17] "ranking_overall"          "lphkm_city"              
[19] "ranking_class"            "lphkm_hwy"               
[21] "url.y"                    "model_class"             
[23] "fuel"                     "estimated_fuel_cost_year"

In [13]:
maintenance <- read.csv("data/maintenance_costs_manual_collection.csv")
maintenance$make <- clean_column(maintenance$make)
maintenance$model <- clean_column(maintenance$model)
nrow(maintenance)
colnames(maintenance)

[1] 14

[1] "make"                    "model"                  
 [3] "year"                    "five_year_maintenance"  
 [5] "eight_year_maintenance"  "five_year_depreciation" 
 [7] "eight_year_depreciation" "five_year_fuel"         
 [9] "msrp_lower"              "msrp_upper"             
[11] "predicted_reliability"

In [14]:
df <- left_join(
    df, 
    maintenance, 
    by=c("make" = "make", "model" = "model", "year" = "year")
)  
nrow(df)
colnames(df)

[1] 1099

[1] "domain.x"                 "description"             
 [3] "date_listed"              "url.x"                   
 [5] "make"                     "title.x"                 
 [7] "year"                     "kilometers"              
 [9] "model"                    "price"                   
[11] "age"                      "engine_size_litres"      
[13] "domain.y"                 "cylinders"               
[15] "title.y"                  "transmission"            
[17] "ranking_overall"          "lphkm_city"              
[19] "ranking_class"            "lphkm_hwy"               
[21] "url.y"                    "model_class"             
[23] "fuel"                     "estimated_fuel_cost_year"
[25] "five_year_maintenance"    "eight_year_maintenance"  
[27] "five_year_depreciation"   "eight_year_depreciation" 
[29] "five_year_fuel"           "msrp_lower"              
[31] "msrp_upper"               "predicted_reliability"

In [15]:
head(df)

domain.x,description,date_listed,url.x,make,title.x,year,kilometers,model,price,⋯,fuel,estimated_fuel_cost_year,five_year_maintenance,eight_year_maintenance,five_year_depreciation,eight_year_depreciation,five_year_fuel,msrp_lower,msrp_upper,predicted_reliability
www.kijiji.ca,Selling my 2002 Ford Explorer XLT. I bought another truck and don't need 2 vehicles. Just finished touring the east coast with it and no problems. I have put over $1000 just in parts in this thing in the last year including Complete struts front and rear rear wheel bearings rear upper ball joints front lower ball joints front outer tie rods axle seals front and rear. Interior is in good shape underside is in good shape body is in good shape except for some surface rust on the roof (see pictures) but it doesn't affect anything and you don't see it with the height of the truck. Everything works on it and the 4x4 is great in the winter you'll never get stuck. The check engine light is on (Downstream O2 sensor inexpensive fix) and the airbag light is on (front crash sensor). Vehicle is being sold as is. Please call or text with any questions. OBO.,26-Aug-16,http://www.kijiji.ca/v-cars-trucks/ottawa/2002-ford-explorer-xlt/1193908774,ford,2002 Ford Explorer XLT,2002,"315,000",explorer,2000,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
www.kijiji.ca,Very clean and accident free rav 4. Safety and etest included. Low payment financing available for bad credit. No credit and bankruptcy. Car looks and drives great. ALL SERVICE DONE AT TOYOTA DEALER!! This is a former U.S vehicle. Odometer reads 145292 miles. Ottawa Auto Sales Call or text 613-883-9090 for any info By apointment only please VEHICLE OPTIONS Air Conditioning Cruise control Power locks Power mirrors Power steering Power trunk Remote keyless entry Tilt wheel Power windows Rear window defroster Rear window wiper Tinted glass CD player Premium audio Bucket seats Airbag driver Airbag passenger Alarm Traction control Alloy wheels Tow package,26-Aug-16,http://www.kijiji.ca/v-cars-trucks/ottawa/2007-toyota-rav4-4wd-only-6800-financing-available/1189078130,toyota,2007 Toyota RAV4 4WD Only $6800! Financing Available!!,2007,"232,468",rav4,6800,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
www.kijiji.ca,Very clean crv for sale. Safety and etest are included. 4 wheel drive. Auto. Air. Power group. Low payment financing available for good credit. Bad credit. No credit and bankruptcy cases. Everyone gets approved. This car is ready to drive away today. Prices to sell for only $9500 call or text 613-883-9090 for any info Ottawa Auto Sales By apointment only please VEHICLE OPTIONS Air Conditioning Cruise control Power locks Power mirrors Power steering Power trunk Remote keyless entry Tilt wheel Power windows Rear window defroster Rear window wiper Tinted glass CD player Premium audio Bucket seats Airbag driver Airbag passenger Airbag side Alarm Anti-lock brakes Traction control Alloy wheels,26-Aug-16,http://www.kijiji.ca/v-cars-trucks/ottawa/2009-honda-cr-v-lx-4wd-low-payment-finacncing-available/1191868778,honda,2009 Honda CR-V LX 4WD Low Payment Finacncing Available !!,2009,"171,118",crv,9500,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
www.kijiji.ca,,26-Aug-16,http://www.kijiji.ca/v-cars-trucks/ottawa/2013-bmw-x3-xdrive28i-navigation-panor-roof/1193918345,bmw,2013 BMW X3 xDrive28i NAVIGATION PANOR. ROOF.,2013,"93,145",x3,28888,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
www.kijiji.ca,Luxury Clean SUV Black on black Accident free Rust free New all season tires Mechanic / Electric A1 no issues at all $4800 as is $5500safety-Etested,26-Aug-16,http://www.kijiji.ca/v-cars-trucks/ottawa/04-bmw-x5/1193912371,bmw,04 BMW X5,2004,240,x5,4800,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
www.kijiji.ca,As is no safety no etest. Vehicle needs some work it was used a work vehicle. AC does not work engine light is on needs brakes. I will take best offer. Come with winter tires. Has low kilometers.,26-Aug-16,http://www.kijiji.ca/v-cars-trucks/ottawa/2003-jeep-liberty-suv-crossover/1193916457,jeep

In [ ]:
library(ggplot2)
ggplot(df) + geom_point(aes(x=df$age, y=df$price, colour=df$make))

In [ ]:
library(ggplot2)
tmp <- df[df$make == "toyota", ]
ggplot(tmp) + geom_point(aes(x=tmp$kilometers, y=tmp$price)) + ggtitle("price vs kilometers")

In [ ]:
write.csv(df, file="data/output.csv")